<a href="https://colab.research.google.com/github/newmantic/Bayesian_options_hedge/blob/main/Bayesian_options_hedge_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yfinance numpy pandas matplotlib scikit-learn scikit-optimize

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np

# Download historical data for a stock and the VIX (Volatility Index)
stock_data = yf.download('SPY', start='2015-01-01', end='2023-01-01')
vix_data = yf.download('^VIX', start='2015-01-01', end='2023-01-01')

# Prepare the dataset
data = pd.DataFrame()
data['Stock_Price'] = stock_data['Close']
data['VIX'] = vix_data['Close']
data.dropna(inplace=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
from math import exp
from scipy.stats import norm

# Black-Scholes model for option pricing
def black_scholes(S, K, T, r, sigma, option_type='put'):
    """
    S: Stock price
    K: Strike price
    T: Time to expiration (in years)
    r: Risk-free interest rate
    sigma: Volatility
    option_type: 'call' or 'put'
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        price = S * norm.cdf(d1) - K * exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        price = K * exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return price

# Simulate portfolio returns with an option hedge
def simulate_portfolio(stock_prices, option_strike, time_to_exp, volatility, r=0.01):
    """
    Simulate the portfolio's performance with a put option hedge.

    stock_prices: Array of historical stock prices.
    option_strike: Strike price of the put option.
    time_to_exp: Time to expiration in years.
    volatility: Volatility of the stock (annualized).
    r: Risk-free interest rate (default 1%).
    """
    portfolio_returns = []
    option_costs = []

    for t in range(1, len(stock_prices)):
        S_t = stock_prices[t]
        S_t_prev = stock_prices[t - 1]
        price_return = (S_t - S_t_prev) / S_t_prev

        # Calculate option price using Black-Scholes
        option_price = black_scholes(S_t_prev, option_strike, time_to_exp, r, volatility, option_type='put')

        # Hedge with the option
        if S_t < option_strike:
            # Hedge payout if stock price falls below strike
            hedge_return = (option_strike - S_t) / option_strike
        else:
            hedge_return = 0

        # Calculate portfolio return with hedge
        portfolio_return = price_return + hedge_return
        portfolio_returns.append(portfolio_return)
        option_costs.append(option_price)

    return np.mean(portfolio_returns), np.mean(option_costs)

In [9]:
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args

# Define the search space for Bayesian Optimization
search_space = [
    Real(0.9, 1.1, name='strike_price_factor'),  # Strike price as a percentage of current price (0.9 to 1.1)
    Real(0.01, 1.0, name='time_to_exp'),  # Time to expiration (in years, from 1 week to 1 year)
    Real(0.1, 0.5, name='volatility')  # Volatility (annualized, 10% to 50%)
]


# Define the objective function for Bayesian Optimization
def objective_function(params):
    # Extract parameters from the passed 'params' list
    strike_price_factor, time_to_exp, volatility = params

    stock_prices = data['Stock_Price'].values

    # Use the stock price at the beginning as a reference for the strike price
    initial_price = stock_prices[0]
    strike_price = strike_price_factor * initial_price

    # Simulate portfolio performance with the given parameters
    mean_return, mean_option_cost = simulate_portfolio(stock_prices, strike_price, time_to_exp, volatility)

    # Objective: Maximize mean_return - mean_option_cost (higher returns, lower costs)
    return -(mean_return - mean_option_cost)  # We negate since we want to minimize the function

In [10]:
#from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args

# Run Bayesian Optimization
result = gp_minimize(objective_function, search_space, n_calls=30, random_state=42)

# Optimal parameters
print(f"Optimal strike price factor: {result.x[0]:.2f}")
print(f"Optimal time to expiration (years): {result.x[1]:.2f}")
print(f"Optimal volatility: {result.x[2]:.2f}")

# Minimum value of the objective function (negative of the profit)
print(f"Optimal portfolio performance: {-result.fun:.4f}")

Optimal strike price factor: 0.90
Optimal time to expiration (years): 0.01
Optimal volatility: 0.10
Optimal portfolio performance: -0.0020
